In [21]:
# imports
import numpy as np
import matplotlib.pyplot as plt

In [22]:
# debugging
import threading
import time

# cur_time = 0
# def increment_time(var):
#     var += 1
# timer = threading.Timer(0.100, increment_time(cur_time))

def start_checkpoint(counter):
    tmr = threading.Timer(0.100, increment_time(counter))
    tmr.start()
    return tmr

def stop_checkpoint(counter, tmr):
    tmr.cancel()
    return cur_time - counter

In [23]:
# setup
grid_data = np.random.randint(1, high=10, size=(10, 10))
multithreading = True
# fig, ax = plt.subplots()
# plt.axis('off')
# grid = ax.table(cellText=grid_data, loc='center')

In [24]:
# Heuristic Algorithm Class
class SafatSearchAlgo:

    def __init__(self, grid, multithreading):
        if grid.size == 0:
            raise Exception('Cannot initialize a grid with size 0')

        self.grid = grid
        self.shape = grid.shape
        self.multithreading = multithreading
        self.memory = {'0,0': [grid[0,0], None]}  # {point: [weight_from_origin, previous_point]}
        self.queue = [[0,0]]


    # Help find the algorithm
    def find_shortest_path(self):

        # Step
        while len(self.queue) > 0:

            # get next point in queue
            vis = self.q_get()
            vis_str = self.coord_to_str(vis)
            vis_point = self.mem_get_point(vis_str)

            # add right side to memory and queue
            right = self.get_right_coord(vis)
            if right:
                self.mem_add_point(right, vis_point[0], vis)
                self.q_enqueue(right)

            # add left side to memory and queue
            bottom = self.get_bottom_coord(vis)
            if bottom:
                self.mem_add_point(bottom, vis_point[0], vis)
                self.q_enqueue(bottom)


    # Backtrack -> returns path in reverse
    def backtrack_path(self):

        # init target info
        grid_shape = self.grid.shape
        target_coord = [grid_shape[0]-1, grid_shape[1]-1]
        path = [target_coord]

        ptr = self.mem_get_point(self.coord_to_str(target_coord))

        # start from target [bottom right] backtrack via 'prev' until [0,0]
        while ptr[1]:
            path.append(ptr[1])
            ptr = self.mem_get_point(self.coord_to_str(ptr[1]))

        return path
    

    # Helper functions
    def coord_to_str(self, coordinates):
        return str(coordinates[0]) + ',' + str(coordinates[1])

    def str_to_coord(self, str_coordinates):
        return [int(x) for x in str_coordinates.split(',')]

    def get_col(self, c):
        return self.grid[c[0], c[1]]
  
    def get_right_coord(self, c):
        if c[1] + 1 < self.grid.shape[1]:
            return [c[0], c[1]+1]

    def get_bottom_coord(self, c):
        if c[0] + 1 < self.grid.shape[0]:
            return [c[0]+1, c[1]]

    def q_enqueue(self, c):
        self.queue.append(c)

    def q_get(self):
        next = self.queue[0]
        self.queue.pop(0)
        return next

    def mem_get_point(self, k):
        try:
            return self.memory[k]
        except KeyError:
            return None

    def mem_add_point(self, coord, prev_weight, prev):
        coord_str = self.coord_to_str(coord)
        wfo = prev_weight+self.get_col(coord)
        point = self.mem_get_point(coord_str)

        # if a point is not in memory or the wfo is less than current path -> add/replace point
        if point is None or point[0] > wfo:
            self.memory[coord_str] = [wfo, prev]


# Dijkstra Algorithm Class

In [25]:
print(grid_data)

algo = SafatSearchAlgo(grid_data, False)
algo.find_shortest_path()
print(algo.memory)
print("Path in reverse:", algo.backtrack_path())